# Лабораторная 1

In [1]:
from Bio import SeqIO, Entrez, SeqUtils, BiopythonParserWarning, Seq, BiopythonWarning
from Bio.PDB.PDBExceptions import PDBConstructionWarning
import warnings
from bs4 import BeautifulSoup
import re


Entrez.email = 'fuck'
warnings.filterwarnings("ignore", category=BiopythonWarning)
warnings.filterwarnings("ignore", category=BiopythonParserWarning)
warnings.filterwarnings("ignore", category=PDBConstructionWarning)

## 1. Подсчитать GC-content для гена с идентификатором X53950.1 (последовательность взять в GeneBank).

In [2]:
with Entrez.efetch(db='nucleotide', id='X53950.1', rettype='gb') as f:
    seq_record = SeqIO.read(f, 'gb')
SeqUtils.GC(seq_record.seq)

35.73213393303348

## 2. Получить последовательность fasta из pdb файла (файл – frompdb.pdb), каждая цепь (последовательность) записывается отдельно со своим заголовком.

In [3]:
SeqIO.convert("frompdb.pdb", "pdb-atom", "frompdb.fasta", "fasta")
!cat frompdb.fasta

>????:A
ADVQQKLAELERQSGGRLGVALINTADNSQIXLYRADERFPMCSTSKVMAAAAVLKQSET
QKQLLNQPVEIKPADLVNYNPIAEKHVNGTMTLAELSAAALQYSDNTAMNKLIAQLGGPG
GVTAFARAIGDETFRLDRTEPTLNTAIPGDPRDTTTPRAMAQTLRQLTLGHALGETQRAQ
LVTWLKGNTTGAASIRAGLPASWVVGDKTGSGXGYGTTNDIAVIWPKDRAPLILVTYFTQ
PQPKAESRRDVLASAAKIVTDG
>????:B
ADVQQKLAELERQSGGRLGVALINTADNSQIXLYRADERFPMCSTSKVMAAAAVLKQSET
QKQLLNQPVEIKPADLVNYNPIAEKHVNGTMTLAELSAAALQYSDNTAMNKLIAQLGGPG
GVTAFARAIGDETFRLDRTEPTLNTAIPGDPRDTTTPRAMAQTLRQLTLGHALGETQRAQ
LVTWLKGNTTGAASIRAGLPASWVVGDKTGSGXGYGTTNDIAVIWPKDRAPLILVTYFTQ
PQPKAESRRDVLASAAKIVTDG


## 3. Перевести ДНК последовательность в РНК, а потом в белок (файл dna.txt)

In [4]:
dna = Seq.Seq(open('dna.txt').read()[:-1])
dna

Seq('GGGATGGCAATAACCCCCCGTTTCTACTTCTAGAGGAGAAAAGTATTGACATGA...GAT')

In [5]:
rna = dna.transcribe()
rna

Seq('GGGAUGGCAAUAACCCCCCGUUUCUACUUCUAGAGGAGAAAAGUAUUGACAUGA...GAU')

In [6]:
kek = rna.translate()
kek.split('*')[:5]

[Seq('GMAITPRFYF'),
 Seq('RRKVLT'),
 Seq('ALPAQGPKKSPISYFRMTCVSLRVNHRPQFIEAWGNR'),
 Seq('V'),
 Seq('LA')]

## 4. Сделать url запрос в GeneBank по идентификатору гена 7157, распарсить полученный файл, чтобы найти информацию: идентификатор в Uniprot кодируемого этим геном белка.

In [7]:
with Entrez.efetch(db='gene', id='7157', rattype='gb', retmode='xml') as f:
    soup = BeautifulSoup(f, 'xml')
soup(string=re.compile('uniprot'))[0].split('/')[-1]

'P04637'

## 5. Поиск мотивов в последовательности с помощью регулярных выражений (dna.txt).
Найти все мотивы (указать индексы начала мотивов в последовательности):
*    __GCNGC__, где N означает любой нуклеотид – сайт, который находит BisI restriction enzyme.
*    Определить эукариотическую мРНК:
ATG – стартовый кодон.
За стартовым кодоном следует последовательность из нуклеотидов (A, T, G или C) длиной от 30 до 1000
Затем следует __полиА-хвост__ (последовательность, состоящая только из А нуклеотидов) длиной от 5 до 10


In [8]:
[x.start() for x in re.finditer('GC.GC', str(dna))]

[296, 580, 1101, 1562, 1721, 1779]

In [9]:
[x.start() for x in re.finditer('ATG[ATGC]{30,1000}A{5,10}', str(dna))]

[102]